**Dưới đây là link google colab phương pháp Memory-Base Collaborative-filtering** [https://colab.research.google.com/drive/1-rEwcF75-wjXT-yHLtrF9nJSdq1lok-H?hl=vi#scrollTo=mO72BGYfiv_x](https://colab.research.google.com/drive/1-rEwcF75-wjXT-yHLtrF9nJSdq1lok-H?hl=vi#scrollTo=mO72BGYfiv_x)

**Import các thư viện cần thiết**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns

**Đọc data các bảng ratings, users, movies từ các file csv**

In [ ]:
ratings = pd.read_csv('ratings.csv', encoding='latin-1', usecols=['userId', 'movieId', 'rating'])
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])
movies = pd.read_csv('movies.csv', encoding='latin-1', usecols=['movieId', 'title', 'genres'])

In [ ]:
ratings.shape

(40022, 3)

**Show data ratings demo**

In [ ]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


**Ma trận User-Item với row là mỗi userId và columns sẽ là tên title của movie đó**
*   Ma trận này có rất nhiều các giá trị miss. Nhiệm vụ của hệ thống là dựa vào các ô đã có giá trị trong ma trận trên (dữ liệu thu được từ trong quá khứ), thông qua mô hình đã được xây dựng, dự đoán các ô còn trống (của user hiện hành). DataFrame chứa đầy xếp hạng mà người dùng đã đưa ra (nếu nó tồn tại, 0 nếu không) 

In [ ]:
user_item_matrix = ratings.pivot('userId','movieId','rating').fillna(0)

In [ ]:
print(f'Shape: {user_item_matrix.shape}')

Shape: (274, 6222)


In [ ]:
user_item_matrix.iloc[:10,:15].astype('i1').T.join(movies.set_index('movieId').title).set_index('title').T.rename_axis('userId')

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),"American President, The (1995)",Dracula: Dead and Loving It (1995),Balto (1995),Nixon (1995),Cutthroat Island (1995)
userId,,,,,,,,,,,,,,,
1,4,0,4,0,0,4,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,4,5,3,5,4,4,3,0,3,4,0,3,0,4
7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,4,0,0,0,0,0,0,0,2,4,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
X_user = cosine_similarity(user_item_matrix)

In [ ]:
X_user.shape

(274, 274)

In [ ]:
X_user[:10,:8].round(3)

array([[1.   , 0.027, 0.06 , 0.194, 0.129, 0.128, 0.159, 0.137],
       [0.027, 1.   , 0.   , 0.004, 0.017, 0.025, 0.028, 0.027],
       [0.06 , 0.   , 1.   , 0.002, 0.005, 0.004, 0.   , 0.005],
       [0.194, 0.004, 0.002, 1.   , 0.129, 0.088, 0.115, 0.063],
       [0.129, 0.017, 0.005, 0.129, 1.   , 0.3  , 0.108, 0.429],
       [0.128, 0.025, 0.004, 0.088, 0.3  , 1.   , 0.076, 0.37 ],
       [0.159, 0.028, 0.   , 0.115, 0.108, 0.076, 1.   , 0.115],
       [0.137, 0.027, 0.005, 0.063, 0.429, 0.37 , 0.115, 1.   ],
       [0.064, 0.   , 0.   , 0.011, 0.   , 0.014, 0.099, 0.   ],
       [0.017, 0.067, 0.   , 0.031, 0.031, 0.02 , 0.132, 0.024]])

**Dự doán rating**

> Ta sẽ dự đoán ratings của một user với mỗi item dựa trên k users gần nhất (neighbor users). Trong đó, N(u, i) là tập k users gần nhất (có độ tương đồng cao nhất) với user u và đã từng đánh giá item i.

> Thực hiện dự đoán cho các trường hợp missing ratings (chưa có dự đoán), ta sẽ thu được ma trận normalized ratings matrix như ví dụ: Cuối cùng, cộng lại các giá trị ratings với ratings trung bình (ở bước chuẩn hóa) theo từng cột. Chúng ta sẽ thu được ma trận hoàn thiện.





In [ ]:
X_item = cosine_similarity(user_item_matrix.T)

In [ ]:
X_item.shape

(6222, 6222)

In [ ]:
X_item[:10,:11].round(2)

array([[1.  , 0.38, 0.31, 0.  , 0.29, 0.37, 0.26, 0.14, 0.25, 0.38, 0.27],
       [0.38, 1.  , 0.3 , 0.09, 0.23, 0.29, 0.2 , 0.17, 0.04, 0.39, 0.29],
       [0.31, 0.3 , 1.  , 0.14, 0.41, 0.34, 0.54, 0.33, 0.35, 0.24, 0.28],
       [0.  , 0.09, 0.14, 1.  , 0.23, 0.15, 0.3 , 0.22, 0.  , 0.12, 0.18],
       [0.29, 0.23, 0.41, 0.23, 1.  , 0.26, 0.51, 0.34, 0.12, 0.25, 0.38],
       [0.37, 0.29, 0.34, 0.15, 0.26, 1.  , 0.24, 0.14, 0.14, 0.36, 0.36],
       [0.26, 0.2 , 0.54, 0.3 , 0.51, 0.24, 1.  , 0.31, 0.15, 0.22, 0.37],
       [0.14, 0.17, 0.33, 0.22, 0.34, 0.14, 0.31, 1.  , 0.  , 0.23, 0.21],
       [0.25, 0.04, 0.35, 0.  , 0.12, 0.14, 0.15, 0.  , 1.  , 0.06, 0.  ],
       [0.38, 0.39, 0.24, 0.12, 0.25, 0.36, 0.22, 0.23, 0.06, 1.  , 0.31]])

Điều này sẽ tạo ra một similarity matrix, với hình dạng (n_users, n_users)

**Thuật toán recommend**
> Trong trường hợp đề xuất dựa trên người dùng, chúng tôi muốn tìm người dùng tương tự với người dùng mới cho người  muốn giới thiệu phim và vì chúng đã có điểm số tương tự, ta phải tìm kiếm các giá trị cao nhất trong một hàng nhất định.

> Dưới đây là định nghĩa function để thực hiện đề xuất 
**User-based Collaborative-filtering and Item-based Collaborative-filtering**




In [ ]:
def movie_recommender(user_item_m, X_user, user, k=20, top_n=10):
    # Get location of the actual movie in the User-Items matrix
    user_location = user_item_m.index.get_loc(user)
    # Use it to index the User similarity matrix
    user_similarities = X_user[user_location]
    # Obtain the indices of the top k most similar users
    most_similar_users = user_item_m.index[user_similarities.argpartition(-k)[-k:]]
    # Obtain the mean ratings of those users for all movies
    rec_movies = user_item_m.loc[most_similar_users].mean(0).sort_values(ascending=False)
    # Discard already seen movies
    m_seen_movies = user_item_m.loc[user].gt(0)
    seen_movies = m_seen_movies.index[m_seen_movies].tolist()
    rec_movies = rec_movies.drop(seen_movies).head(top_n)
    # Return recommendations - top similar users rated movies
    return rec_movies.index.to_frame().reset_index(drop=True).merge(movies)

In [ ]:
class CfRec():
    def __init__(self, M, X, items, k=20, top_n=10):
        self.X = X
        self.M = M
        self.k = k
        self.top_n = top_n
        self.items = items
        
    def recommend_user_based(self, user):
        ix = self.M.index.get_loc(user)
        # Use it to index the User similarity matrix
        u_sim = self.X[ix]
        # obtain the indices of the top k most similar users
        most_similar = self.M.index[u_sim.argpartition(-(self.k+1))[-(self.k+1):]]
        # Obtain the mean ratings of those users for all movies
        rec_items = self.M.loc[most_similar].mean(0).sort_values(ascending=False)
        # Discard already seen movies
        # already seen movies
        seen_mask = self.M.loc[user].gt(0)
        seen = seen_mask.index[seen_mask].tolist()
        rec_items = rec_items.drop(seen).head(self.top_n)
        # return recommendations - top similar users rated movies
        return (rec_items.index.to_frame()
                                .reset_index(drop=True)
                                .merge(self.items))

    def recommend_item_based(self, item):
        liked = self.items.loc[self.items.movieId.eq(item), 'title'].item()
        print(f"Because you liked {liked}, we'd recommend you to watch:")
        # get index of movie
        ix = self.M.columns.get_loc(item)
        # Use it to index the User similarity matrix
        i_sim = self.X[ix]
        # obtain the indices of the top k most similar users
        most_similar = self.M.columns[i_sim.argpartition(-(self.k+1))[-(self.k+1):]]
        return (most_similar.difference([item])
                                 .to_frame()
                                 .reset_index(drop=True)
                                 .merge(self.items)
                                 .head(self.top_n))

In [ ]:
def because_user_liked(user_item_matrix, movies, ratings, user):
    ix_user_seen = user_item_matrix.loc[user]>0.
    seen_by_user = user_item_matrix.columns[ix_user_seen]
    return (seen_by_user.to_frame()
                 .reset_index(drop=True)
                 .merge(movies)
                 .assign(userId=user)
                 .merge(ratings[ratings.userId.eq(user)])
                 .sort_values('rating', ascending=False).head(10))

Trước khi test bộ recommender. Chúng ta sẽ sắp xếp các rating được đưa ra cho các bộ phim được người dùng nhìn thấy và lấy 10 người đầu tiên

**Testing the User-based Collaborative-filtering và đưa ra đề xuất**

In [ ]:
recommender = CfRec(user_item_matrix, X_user, movies)

**Dưới đây là những bộ phim được người dùng có userId là 69 đánh giá cao nhất**

In [ ]:
because_user_liked(user_item_matrix, movies, ratings, 69)

,movieId,title,genres,userId,rating
0,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,69,5.0
14,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,69,5.0
44,4880,Life as a House (2001),Drama,69,5.0
42,4874,K-PAX (2001),Drama|Fantasy|Mystery|Sci-Fi,69,5.0
36,4027,"O Brother, Where Art Thou? (2000)",Adventure|Comedy|Crime,69,5.0
29,3671,Blazing Saddles (1974),Comedy|Western,69,5.0
28,3552,Caddyshack (1980),Comedy,69,5.0
27,3175,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi,69,5.0
26,2951,"Fistful of Dollars, A (Per un pugno di dollari...",Action|Western,69,5.0
25,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery,69,5.0


**Dưới đây là Output của phương pháp User-based Collaborative-filtering với userId là 69**

In [ ]:
recommender.recommend_user_based(69)

,movieId,title,genres
0,858,"Godfather, The (1972)",Crime|Drama
1,2858,American Beauty (1999),Drama|Romance
2,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
3,1270,Back to the Future (1985),Adventure|Comedy|Sci-Fi
4,2028,Saving Private Ryan (1998),Action|Drama|War
5,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
6,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
7,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
8,1221,"Godfather: Part II, The (1974)",Crime|Drama
9,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi


**Testing the Item-based Collaborative-filtering và đưa ra đề xuất**

In [ ]:
recommender = CfRec(user_item_matrix, X_item, movies)

**Dưới đây là những bộ phim được người dùng có userId là 100 đánh giá cao nhất**

In [68]:
because_user_liked(user_item_matrix, movies, ratings, 100)

,movieId,title,genres,userId,rating
86,1958,Terms of Endearment (1983),Comedy|Drama,100,5.0
101,2423,Christmas Vacation (National Lampoon's Christm...,Comedy,100,5.0
137,5620,Sweet Home Alabama (2002),Comedy|Romance,100,5.0
55,1101,Top Gun (1986),Action|Romance,100,5.0
125,4041,"Officer and a Gentleman, An (1982)",Drama|Romance,100,5.0
70,1307,When Harry Met Sally... (1989),Comedy|Romance,100,4.5
84,1912,Out of Sight (1998),Comedy|Crime|Drama|Romance|Thriller,100,4.5
82,1777,"Wedding Singer, The (1998)",Comedy|Romance,100,4.5
81,1680,Sliding Doors (1998),Drama|Romance,100,4.5
80,1678,"Joy Luck Club, The (1993)",Drama|Romance,100,4.5


**Dưới đây là Output của phương pháp Item-based Collaborative-filtering với userId là 100**

In [69]:
recommender.recommend_item_based(100)

Because you liked City Hall (1996), we'd recommend you to watch:


,movieId,title,genres
0,26,Othello (1995),Drama
1,45,To Die For (1995),Comedy|Drama|Thriller
2,61,Eye for an Eye (1996),Drama|Thriller
3,78,"Crossing Guard, The (1995)",Action|Crime|Drama|Thriller
4,113,Before and After (1996),Drama|Mystery
5,191,"Scarlet Letter, The (1995)",Drama|Romance
6,194,Smoke (1995),Comedy|Drama
7,270,Love Affair (1994),Drama|Romance
8,302,"Queen Margot (Reine Margot, La) (1994)",Drama|Romance
9,354,Cobb (1994),Drama
